# WPILib ML Notebook


## Introduction

By using this notebook, you can train a TensorFlow Lite model for use on a Raspberry Pi and Google Coral USB Accelerator. We've designed this process to be as simple as possible. If you find an issue with this notebook, please create a new issue report on our [GitHub page](https://github.com/wpilibsuite/CoralSagemaker), where you downloaded this notebook.

### Training

#### Complete instructions can be found [here](https://github.com/wpilibsuite/CoralSagemaker/blob/master/docs/training.md).

## Notebook

In [ ]:
from sagemaker.estimator import Estimator
from sagemaker import get_execution_role


# Uses GPU by default, change to false to use CPU
use_gpu = True

role = get_execution_role()

instance_type = None
algorithm_name = None

if use_gpu:
    instance_type = 'ml.p3.2xlarge'
    algorithm_name = 'wpi-gpu'
else:
    instance_type = 'ml.c4.2xlarge'
    algorithm_name = 'wpi-cpu'


"""
Hyperparameters:
    epochs -> int: number of training steps. Training time is proportional to this number. default = 1000
    batch_size -> int: size of a batch of training images. default = 32
"""
hyperparameters = {'epochs': 1000,
                   'batch_size': 32}

ecr_image = "249838237784.dkr.ecr.us-east-1.amazonaws.com/{}:latest".format(algorithm_name)

# The estimator object, using our notebook, training instance, the ECR image, and the specified training steps
estimator = Estimator(role=role,
                      train_instance_count=1,
                      train_instance_type=instance_type,
                      image_name=ecr_image,
                      hyperparameters=hyperparameters)

# Change this bucket if you want to train with your own data. The WPILib bucket contains thousands of high quality labeled images.
# s3://wpilib
estimator.fit("s3://wpilib")